<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_Ciencia_de_Dados/blob/main/IMT_Pipeline_Classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# Pipeline de Classificação

Aqui você vai encontrar um roteiro bastante completo para tratar problemas de classificação. Isso inclui:

1. Tratamento de dados ausentes
2. Seleção de Features
3. Label e Hot encode dos dados
4. Normalização
5. Seleção entre diferentes modelos e por diferentes métricas
6. Análise dos Resultados
7. Predição de novos casos (reaplicando as transformações do modelo)

Além de uma discussão sobre a eficiência de modelos em casos reais como este.

Ao final você ainda vai entender que o método que aplicamos ainda precisa ser refinado para construção de um pipeline completo de classificação, e é o que você irá aprender a seguir.

# Imports

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Aquisição dos dados

In [77]:
df = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/blood_risk.csv')
df.head()

,record,name,cpf,birthday,Gluconise,Adrino_posina,Gamma_R,Nitro_K,H278,Plate,Factor_risk,norm_gamma_type,Blood_type,Blood_Rh,Low_Risk,Medium_Risk,High_Risk
0,89256,Bryan Cardoso,83619570221,2006-11-14,916.6484,Uncovered,2,3.07,5749.42,DD,5,1,AB,+,True,False,False
1,72173,Sr. Vicente Lima,30861597230,2013-11-22,988.7466,Clinical Done,5,2.53,285.18,NaN,2,1,O,+,True,False,False
2,95816,Otávio da Rosa,89532614737,2008-10-23,572.7608,Uncovered,4,3.30,8000.08,X,5,1,O,-,True,False,False
3,44566,Alícia da Cruz,53027691406,2011-07-25,1328.0542,Clinical Done,5,3.07,5737.50,DD,5,1,AB,-,False,True,True
4,31019,Maysa Novaes,53817096259,2021-06-25,641.3129,Clinical Done,3,3.05,5476.12,B,5,1,NaN,+,False,True,True


# Tratamento de nulos

As principais alternativas para o tratamento de valores nulos consiste na:

1. Exclusão do casos nulos (`df.dropna()`)
2. Exclusão de atributos nulos (`df.drop(columns='coluna')`)
3. Imputar os dados nulos

Aqui optamos por imputar dados ausentes empregando a mediana ou a moda, e como você verá mais adiante isso é, em geral, mais aplicável quando dados ausentes poderão ocorrer também em novos casos para predição.

In [78]:
df.isnull().sum() / len(df)

record             0.000000
name               0.000000
cpf                0.000000
birthday           0.000000
Gluconise          0.010262
Adrino_posina      0.144567
Gamma_R            0.000000
Nitro_K            0.005231
H278               0.005634
Plate              0.112978
Factor_risk        0.000000
norm_gamma_type    0.000000
Blood_type         0.139638
Blood_Rh           0.000000
Low_Risk           0.000000
Medium_Risk        0.000000
High_Risk          0.000000
dtype: float64

In [79]:
1 - len(df.dropna()) / len(df)

0.360261569416499

In [80]:
df.select_dtypes('number').isnull().sum()

record               0
cpf                  0
Gluconise          102
Gamma_R              0
Nitro_K             52
H278                56
Factor_risk          0
norm_gamma_type      0
dtype: int64

In [81]:
for c in df.select_dtypes('number'):
  if df[c].isnull().sum() > 0:
    df[c] = df[c].fillna(df[c].median())

df.select_dtypes('number').isnull().sum()

record             0
cpf                0
Gluconise          0
Gamma_R            0
Nitro_K            0
H278               0
Factor_risk        0
norm_gamma_type    0
dtype: int64

In [82]:
df.select_dtypes(exclude='number').isnull().sum()

name                0
birthday            0
Adrino_posina    1437
Plate            1123
Blood_type       1388
Blood_Rh            0
Low_Risk            0
Medium_Risk         0
High_Risk           0
dtype: int64

In [83]:
for c in df.select_dtypes(exclude='number'):
  if df[c].isnull().sum() > 0:
    df[c] = df[c].fillna(df[c].mode()[0])

df.select_dtypes(exclude='number').isnull().sum()

name             0
birthday         0
Adrino_posina    0
Plate            0
Blood_type       0
Blood_Rh         0
Low_Risk         0
Medium_Risk      0
High_Risk        0
dtype: int64

## Se optar por excluir os valores ausentes

In [84]:
%%script --no-raise-error false

df = df.dropna()
df = df.reset_index(drop=True)

# Tratamento do atributo `birthday`

Atributos novos ou transformados são normalmente denominados **atributos derivados**, com a idade a partir de uma data de nascimento, a operadora do cartão a partir do número do cartão ou a cidade a partir do prefixo do número, uma distância ao centro de distribuição mais próximo a partir da localização, a taxa do dólar a partir de uma data ou ainda a temperatura ou a presença de chuva para um dado dia.

In [85]:
df.birthday = df.birthday.apply(lambda x: int(x[:4]))
df.birthday

0       2006
1       2013
2       2008
3       2011
4       2021
        ... 
9935    2022
9936    2015
9937    2014
9938    2013
9939    2004
Name: birthday, Length: 9940, dtype: int64

# Tratamento do atributo `target`  

Os valores, apesar da aparente inconsistência, produz classes mutuamente exclusivas.

In [86]:
df[['Low_Risk','Medium_Risk','High_Risk']].value_counts()

Low_Risk  Medium_Risk  High_Risk
True      False        False        4944
False     True         False        3338
                       True         1658
dtype: int64

In [87]:
df['Risk'] = 1*df['Low_Risk'].astype('int')  + 2*df['Medium_Risk'].astype('int') + 3*df['High_Risk'].astype('int')
df['Risk'] = df['Risk'].replace([1,2,5],['Low','Medium','High'])

df.head()

,record,name,cpf,birthday,Gluconise,Adrino_posina,Gamma_R,Nitro_K,H278,Plate,Factor_risk,norm_gamma_type,Blood_type,Blood_Rh,Low_Risk,Medium_Risk,High_Risk,Risk
0,89256,Bryan Cardoso,83619570221,2006,916.6484,Uncovered,2,3.07,5749.42,DD,5,1,AB,+,True,False,False,Low
1,72173,Sr. Vicente Lima,30861597230,2013,988.7466,Clinical Done,5,2.53,285.18,DD,2,1,O,+,True,False,False,Low
2,95816,Otávio da Rosa,89532614737,2008,572.7608,Uncovered,4,3.30,8000.08,X,5,1,O,-,True,False,False,Low
3,44566,Alícia da Cruz,53027691406,2011,1328.0542,Clinical Done,5,3.07,5737.50,DD,5,1,AB,-,False,True,True,High
4,31019,Maysa Novaes,53817096259,2021,641.3129,Clinical Done,3,3.05,5476.12,B,5,1,AB,+,False,True,True,High


# Exclusão de atributos

Atibutos identificadores, com alta correlação ou não preditores devem ser eliminados.

In [88]:
df.corr() > 0.9

<ipython-input-88-8bdfe56fe53f>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr() > 0.9


,record,cpf,birthday,Gluconise,Gamma_R,Nitro_K,H278,Factor_risk,norm_gamma_type,Low_Risk,Medium_Risk,High_Risk
record,True,False,False,False,False,False,False,False,False,False,False,False
cpf,False,True,False,False,False,False,False,False,False,False,False,False
birthday,False,False,True,False,False,False,False,False,False,False,False,False
Gluconise,False,False,False,True,False,False,False,False,False,False,False,False
Gamma_R,False,False,False,False,True,False,False,False,False,False,False,False
Nitro_K,False,False,False,False,False,True,True,False,False,False,False,False
H278,False,False,False,False,False,True,True,False,False,False,False,False
Factor_risk,False,False,False,False,False,False,False,True,False,False,False,False
norm_gamma_type,False,False,False,False,False,False,False,False,True,False,False,False
Low_Risk,False,False,False,False,False,False,False,False,False,True,False,False


In [89]:
df = df.drop(columns=['record','name','cpf','Nitro_K','Low_Risk','Medium_Risk','High_Risk'])

df.head()

,birthday,Gluconise,Adrino_posina,Gamma_R,H278,Plate,Factor_risk,norm_gamma_type,Blood_type,Blood_Rh,Risk
0,2006,916.6484,Uncovered,2,5749.42,DD,5,1,AB,+,Low
1,2013,988.7466,Clinical Done,5,285.18,DD,2,1,O,+,Low
2,2008,572.7608,Uncovered,4,8000.08,X,5,1,O,-,Low
3,2011,1328.0542,Clinical Done,5,5737.50,DD,5,1,AB,-,High
4,2021,641.3129,Clinical Done,3,5476.12,B,5,1,AB,+,High


# Hot encode

Quando empregar Hot Encode ou Label Encode para transformação de atributos categóricos?

*  **Hot Encode**. No máximo dezenas valores categóricos e requerido para modelos que trabalham com distância ou empregam diretamente os valores com K-Vizinhos mais Próximos, Regressão Logística e Linear.

*  **Label Encode**. Pode ser aplicada para vários valores categóricos, mas implica em uma ordem dos valores devendo ser evitado para modelos que trabalham com distância ou empregam diretamente os valores. Pode, entretanto, ser aplicada sem problemas para modelos que empregam a probabilidade dos valores no lugar dos valores diretamente, como modelos de Árvores de Decisão, Naive Bayes e Random Forest.

**Dica:** 1. Sempre que possível opte pelo Hot Encode, mesmo que vá empregar um modelo de Árvore de Decisão, por ser mais flexível. 2. Inviável aplicar para textos onde técnicas de **Word Embedding Encoding** como `word2vec` são mais adequadas.

In [90]:
from sklearn.preprocessing import OneHotEncoder

hot_encode = OneHotEncoder(handle_unknown='ignore',sparse_output=False,drop='first')
hot_encode.fit(df.drop(columns='Risk').select_dtypes(exclude='number'))

df_hot_encode = pd.DataFrame(hot_encode.transform(df.drop(columns='Risk').select_dtypes(exclude='number')),columns=hot_encode.get_feature_names_out())
df_hot_encode.head()

df = pd.concat([df_hot_encode,df.select_dtypes('number'),df[['Risk']]],axis=1)
df.head()

,Adrino_posina_Clinical Done,Adrino_posina_Covered,Adrino_posina_Uncovered,Plate_B,Plate_CS,Plate_DD,Plate_X,Blood_type_AB,Blood_type_B,Blood_type_O,Blood_Rh_-,birthday,Gluconise,Gamma_R,H278,Factor_risk,norm_gamma_type,Risk
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2006,916.6484,2,5749.42,5,1,Low
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2013,988.7466,5,285.18,2,1,Low
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2008,572.7608,4,8000.08,5,1,Low
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2011,1328.0542,5,5737.50,5,1,High
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2021,641.3129,3,5476.12,5,1,High


## Se optar pelo `LabelEncoder`

In [91]:
%%script --no-raise-error false
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

df_labels = df.drop(columns='Risk').select_dtypes(exclude='number').apply(LabelEncoder().fit_transform)

df = pd.concat([df_labels, df.select_dtypes('number'), df[['Risk']]], axis=1)
df.head()

# Normalização

Modelos baseados em distância como K-vizinhos mais próximos e regressão logística, são sensíveis à normalização, devendo nesses casos sempre ser empregada. Modelos baseados em probabilidades podem ser empregados sem a normalização, mas a normalização, em geral, aumenta a explicabilidade dos modelos.

Dica: Empregue `StandardScaler` ou `MinMaxScaler`.

In [92]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
scaler.fit(df.drop(columns='Risk'))

df_scaled = scaler.transform(df.drop(columns='Risk'))
df_scaled = pd.DataFrame(df_scaled, columns=df.drop(columns='Risk').columns)

df_scaled = pd.concat([df_scaled,df[['Risk']]],axis=1)

df = df_scaled
df.head()

,Adrino_posina_Clinical Done,Adrino_posina_Covered,Adrino_posina_Uncovered,Plate_B,Plate_CS,Plate_DD,Plate_X,Blood_type_AB,Blood_type_B,Blood_type_O,Blood_Rh_-,birthday,Gluconise,Gamma_R,H278,Factor_risk,norm_gamma_type,Risk
0,-0.608505,-0.466412,1.178738,-0.532050,-0.391321,1.307076,-0.501729,1.196456,-0.474228,-0.606022,-0.579983,-0.776417,-0.397430,-1.345079,-0.415279,0.021329,0.988198,Low
1,1.643372,-0.466412,-0.848365,-0.532050,-0.391321,1.307076,-0.501729,-0.835802,-0.474228,1.650105,-0.579983,0.257874,-0.035530,0.000903,-0.970086,-1.865980,0.988198,Low
2,-0.608505,-0.466412,1.178738,-0.532050,-0.391321,-0.765067,1.993110,-0.835802,-0.474228,1.650105,1.724187,-0.480905,-2.123588,-0.447758,-0.186760,0.021329,0.988198,Low
3,1.643372,-0.466412,-0.848365,-0.532050,-0.391321,1.307076,-0.501729,1.196456,-0.474228,-0.606022,1.724187,-0.037638,1.667638,0.000903,-0.416489,0.021329,0.988198,High
4,1.643372,-0.466412,-0.848365,1.879522,-0.391321,-0.765067,-0.501729,1.196456,-0.474228,-0.606022,-0.579983,1.439921,-1.779487,-0.896418,-0.443028,0.021329,0.988198,High


# Select Features?

Aplique quando o número de features for excessivo (>100), existem várias métricas e para classificação o `SelectKBest` é o mais comumente empregado. Mas não esqueça as limitações dessa abordagem que é uma abordagem apenas **univariada** do poder explicativo das variáveis preditoras.

In [93]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

X = df.drop(columns=['Risk'])
y = df['Risk']

select_features = SelectKBest(mutual_info_classif, k=10).fit(X, y)
# print( select_features.get_support() )
print( list(X.columns[select_features.get_support()]))

['Adrino_posina_Clinical Done', 'Plate_CS', 'Plate_DD', 'Plate_X', 'Blood_type_B', 'Blood_type_O', 'Blood_Rh_-', 'H278', 'Factor_risk', 'norm_gamma_type']


In [94]:
select_features.scores_

array([5.04412193e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.52983192e-03, 9.73745553e-05, 6.23196835e-03, 0.00000000e+00,
       1.03181474e-02, 1.10420971e-03, 1.05696929e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.06942118e-03, 5.38583697e-04,
       2.69720237e-04])

# Aplicando diferentes modelos

O Hot Encode e a normalização garantem a aplicação dos mesmos dados para diferentes modelos.

## Seleção do Modelos

> Modelos:

```
DecisionTreeClassifier(),
KNeighborsClassifier(),
MLPClassifier(),
```

> Hiperparâmetros

```
DecisionTreeClassifier(criterion='log_loss'),
KNeighborsClassifier(10),
MLPClassifier(max_iter=1000,hidden_layer_sizes=(64,128,64)),

```

A rigor `KNeighborsClassifier(10)` e `KNeighborsClassifier(5)` são *modelos* diferentes, mas nos referimos a isso como hiperparâmetros que podem ser ajustados em um modelo.

In [95]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df.drop(columns='Risk')
y = df['Risk']

base_estimators = [ DecisionTreeClassifier(criterion='log_loss'),
                    DecisionTreeClassifier(criterion='entropy'),
                    RandomForestClassifier(),
                    KNeighborsClassifier(100),
                    KNeighborsClassifier(1),
                    LogisticRegression(max_iter=10000),
                    AdaBoostClassifier(),
                    GradientBoostingClassifier(),
                    GaussianNB(),
                    SVC(),
                    MLPClassifier(max_iter=1000,hidden_layer_sizes=(64,128,64)),
                    ]

best_score = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state = 1)

for estimator in base_estimators:
  clf = estimator
  clf.fit(X_train, y_train)

  score = clf.score(X_test, y_test) # accuracy

  #
  # Substituindo o score para precisão de Risk High
  #
  # report = classification_report(y_test,clf.predict(X_test),output_dict=True)
  # score = report['High']['precision']

  if score > best_score:
    best_score = score
    best_model = clf

    print('\n Best Model: ' , best_model, ' score = ', best_score)





 Best Model:  DecisionTreeClassifier(criterion='log_loss')  score =  0.3876592890677398

 Best Model:  RandomForestClassifier()  score =  0.4584171696847753

 Best Model:  KNeighborsClassifier(n_neighbors=100)  score =  0.4969818913480885


# Analisando os resultados

A acuracidade é a primeira mas uma métrica bastante geral de classificação e outras métricas precisam ser analisadas sempre que encontramos uma acuracidade satisfatória. As mais comuns são a **precisão**, a **revocação** e o **F1-score** (reveja nas notas de aula).

Esses parâmetros também podem ser igualmente empregados para seleção do modelo como no exemplo acima em que empregamos a precisão da classe risco.

In [96]:
print(classification_report(y_test,best_model.predict(X_test)))

              precision    recall  f1-score   support

        High       0.00      0.00      0.00       497
         Low       0.50      0.98      0.66      1483
      Medium       0.40      0.02      0.05      1002

    accuracy                           0.50      2982
   macro avg       0.30      0.34      0.24      2982
weighted avg       0.38      0.50      0.34      2982



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn.metrics import precision_score

precision_score(y_test,best_model.predict(X_test), average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3881449213444565

In [23]:
report = classification_report(y_test,best_model.predict(X_test),output_dict=True)
report.keys()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


dict_keys(['High', 'Low', 'Medium', 'accuracy', 'macro avg', 'weighted avg'])

In [24]:
report['High']

{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 497}

In [25]:
report['High']['precision']

0.0

# Fazendo a predição de novos dados

## Transformações dos dados

In [31]:
def preparacao(df_cases):

  df = df_cases.copy()

  for c in df.select_dtypes('number'):
    if df[c].isnull().sum() > 0:
      df[c] = df[c].fillna(df[c].median())

  for c in df.select_dtypes(exclude='number'):
    if df[c].isnull().sum() > 0:
      df[c] = df[c].fillna(df[c].mode()[0])

  df.birthday = df.birthday.apply(lambda x: int(x[:4]))

  df = df.drop(columns=['record','name','cpf','Nitro_K'])

  df_hot_encode = pd.DataFrame(hot_encode.transform(df.select_dtypes(exclude='number')),columns=hot_encode.get_feature_names_out())

  df = pd.concat([df_hot_encode,df.select_dtypes('number')],axis=1)

  df_scaled = scaler.transform(df)
  df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

  return df

In [32]:
df_cases = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/blood_test2_noanswers.csv')
df_cases.head()

,record,name,cpf,birthday,Gluconise,Adrino_posina,Gamma_R,Nitro_K,H278,Plate,Factor_risk,norm_gamma_type,Blood_type,Blood_Rh
0,85071,Pedro Henrique Araújo,6952437800,2017-08-06,1166.2016,Uncovered,4,2.66,1584.72,DD,3,0,AB,+
1,7206,Sr. Lucca Gomes,47682305126,2018-08-22,1140.6751,Clinical Done,5,2.92,4228.09,CS,4,1,AB,+
2,47727,Davi Luiz da Rosa,30869742574,2017-10-28,848.3981,Capsuled,3,2.90,4069.99,CS,4,1,A,+
3,64626,Vitor Hugo Souza,17693205470,2013-10-05,761.2512,Covered,5,3.47,9763.64,DD,5,1,A,+
4,76250,Srta. Isadora Melo,52874306126,2014-12-02,977.8314,Clinical Done,6,3.09,NaN,DD,5,0,B,+


In [33]:
df_cases = preparacao(df_cases)
df_cases.head()

,Adrino_posina_Clinical Done,Adrino_posina_Covered,Adrino_posina_Uncovered,Plate_B,Plate_CS,Plate_DD,Plate_X,Blood_type_AB,Blood_type_B,Blood_type_O,Blood_Rh_-,birthday,Gluconise,Gamma_R,H278,Factor_risk,norm_gamma_type
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2017,1166.2016,4,1584.72,3,0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2018,1140.6751,5,4228.09,4,1
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,848.3981,3,4069.99,4,1
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2013,761.2512,5,9763.64,5,1
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2014,977.8314,6,7204.38,5,0


In [34]:
df_cases['Risk'] = best_model.predict(df_cases)
df_cases['Risk']

0     Low
1     Low
2     Low
3     Low
4     Low
5     Low
6     Low
7     Low
8     Low
9     Low
10    Low
11    Low
12    Low
13    Low
14    Low
15    Low
16    Low
17    Low
18    Low
19    Low
20    Low
21    Low
22    Low
23    Low
24    Low
25    Low
26    Low
27    Low
28    Low
29    Low
Name: Risk, dtype: object

In [35]:
df_cases_risk = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/blood_test2_answers_short.csv')
df_cases_risk['Risk']

0        Low
1        Low
2       High
3       High
4     Medium
5       High
6        Low
7        Low
8     Medium
9     Medium
10       Low
11    Medium
12       Low
13       Low
14       Low
15       Low
16    Medium
17    Medium
18    Medium
19      High
20       Low
21       Low
22    Medium
23       Low
24       Low
25       Low
26       Low
27    Medium
28      High
29       Low
Name: Risk, dtype: object

In [36]:
sum(df_cases['Risk'] == df_cases_risk['Risk']) / len(df_cases)

0.5333333333333333

# Revisando o problema

O resultado do modelo é bastante pobre embora, 50% seja um valor acima do que uma escolha aleatória poderia oferecer (33%). Não havendo erros de código ou nos procedimentos é necessário voltarmos ao problema, à exploração dos dados etc. (veja a Discussão a seguir).

Duas alternativas frequentemente empregadas baseiam-se nas seguintes afirmativas:

> **1. Existem outros dados que podem ser empregados no modelo?**

> **2. O modelo, mesmo insuficiente, consegue trazer algum tipo de resultado que agrega valor?**

Sem acesso outros neste caso, podemos no perguntar se podemos por exemplo criar um **modelo eficiente ao menos para os casos de maior risco**. É o que você encontrará a seguir.

O resultado, 0.86 parece animador, mas uma análise rápida permitirá você ver que isso corresponde aproximadamente à probabilidade de encontrarmos um paciente com risco alto o que mostra que o modelo simplesmente captura a probabilidade da classe.

In [37]:
df['Risk'] = df['Risk'] == 'High'
df['Risk'].value_counts()

False    8282
True     1658
Name: Risk, dtype: int64

In [51]:
X = df.drop(columns='Risk')
y = df['Risk']

base_estimators = [ DecisionTreeClassifier(criterion='log_loss'),
                    DecisionTreeClassifier(criterion='entropy'),
                    RandomForestClassifier(),
                    KNeighborsClassifier(10),
                    KNeighborsClassifier(1),
#                    LogisticRegression(max_iter=10000),
                    AdaBoostClassifier(),
                    GradientBoostingClassifier(),
#                    GaussianNB(),
#                    SVC()
                    ]

best_score = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state = 1)

for estimator in base_estimators:
  clf = estimator
  clf.fit(X_train, y_train)

  score = clf.score(X_test, y_test) # accuracy

  #
  # Substituindo o score para precisão de Risk High
  #
  # report = classification_report(y_test,clf.predict(X_test),output_dict=True)
  # score = report['True']['precision']

  if score > best_score:
    best_score = score
    best_model = clf

    print('\n Best Model: ' , best_model, ' score = ', best_score)





 Best Model:  DecisionTreeClassifier(criterion='log_loss')  score =  0.7072434607645876

 Best Model:  DecisionTreeClassifier(criterion='entropy')  score =  0.7142857142857143

 Best Model:  RandomForestClassifier()  score =  0.8326626425217974


In [52]:
print(classification_report(y_test,best_model.predict(X_test)))

              precision    recall  f1-score   support

       False       0.83      1.00      0.91      2485
        True       0.25      0.00      0.00       497

    accuracy                           0.83      2982
   macro avg       0.54      0.50      0.46      2982
weighted avg       0.74      0.83      0.76      2982



In [53]:
df_cases = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/blood_test2_noanswers.csv')
df_cases = preparacao(df_cases)
df_cases['Risk'] = best_model.predict(df_cases)
sum(df_cases['Risk'] == ( df_cases_risk['Risk'] == 'High')) / len(df_cases)

0.8333333333333334

# Discussão

Muitos modelos de classificação no livros e exemplos na internet fornecem resultados maravilhosos com acuracidade acima de 0.8 em muitos casos. Na prática, em casos reais, acuracidades dessa ordem são muito mais difíceis de serem alcançadas. Há vários motivos por que isso pode acontecer, e eles são muito mais comuns do que podemos encontrar em bases dados públicas, artigos ou estudos científicos que, em geral, focam em resultados positivos. Os principais são:

1. Classes desbalanceadas (ex. 99% 1%)
2. Custos assimétricos de erro (ex. Benigno Maligno)
3. Dados ruidosos
4. **Dados incompletos**. Esse talvez seja o caso mais geral e o mais difícil de tratarmos. Apesar de termos dados os dados podem não serem suficientes para fazermos predições.










## Exemplos

![imagem](https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/blob/main/images/worst_classification.png?raw=true)

Figura. Piores datasets do UCI para classificação

1. **Stock Market**. Os melhores modelos de predição de alta (1) ou baixa (0) empregando centenas de dados e com dezenas de atributos do mercado (>50) não superam uma acuracidade de 0.55 (para precisão e recall balanceados). As variáveis não se mostram suficientemente preditoras. Por isso, recentemente, muitos modelos buscam incorporar dados externos como o sentimento dos investidores.

2. **Customer Churn, Customer Satisfation**. São problemas em geral com classes bastante desbalanceadas (1-2%, para True), e modelos muito sofisticados podem alcançar uma acuracidade até alta (0.82, ver Soraya_Jimenez, Will Cukierski. (2016). Santander Customer Satisfaction. Kaggle. https://kaggle.com/competitions/santander-customer-satisfaction) mas resultados bastante menores de precisão e revocação.

3. **Spotify**. Prever o gênero de uma música através de dados do Spotify (medidas do som como acousticness, energy ou beats per minute) pode ser
desafiador mesmo tendo dados como o nome da música ou o artista (0.70, Marc Roper. (2021). CS985/6 Spotify Classification Problem 2021. Kaggle. https://kaggle.com/competitions/cs9856-spotify-classification-problem-2021). Mas o problema se torna mais complexo ainda se você tiver um artista novo ou ainda se quiser prever se a música vai se tornar um hit ou não.

4. **Airbnb, Amazon, IFood**. Somente dados tabulares são muitas vezes insuficientes para predições é o caso de vários conjuntos de dados do comércio eletrônico em que dados textuais (recomendações, comentários, descrições do produto ou item) precisam ser empregados. Nesses casos o uso limitado de dados tabulares (excluindo textos que ainda não vimos como tratar) levam, em geral, a resultados bastante insatisfatórios.

Além disso, modelos de classificação estão sujeitos a muitos outros problemas que requerem cuidado. Os modelos de classificação muitas vezes podem ser **tendenciosos, enganados e não serem generalizáveis**.

# Problemas metodológicos

Nossos procedimentos empregados até aqui fornecem um primeiro insight para os modelos, mas podem e precisam ainda ser refinados corrigindo alguns problemas.

**Primeiro Problema**. O primeiro problema é bastante simples de você entender: **empregamos até agora um único conjunto de teste**. Embora aleatório, cada conjunto trará um resultado de acuracidade (e das outras métricas) diferente. Que valor você deve empregar se um conjunto fornece 0.84 e outro resulta em 0.96? O mais razoável seria fazer a média de vários conjuntos de teste diferentes. Mesmo assim, esse número precisaria ser suficientemente grande e não há garantia de que algum conjunto de dados não seja incluído no teste resultando em uma acuracidade que não representa todo o conjunto de dados.




In [110]:
X = df.drop(columns='Risk')
y = df['Risk']

score_array = []

for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

  clf = RandomForestClassifier()
  clf.fit(X_train, y_train)

  score = clf.score(X_test, y_test) # accuracy
  score_array.append(score)

  print('\n Model: ' , clf, ' score = ', score)

print('\n Model: ' , clf, ' mean score = ', np.array(score_array).mean())


 Model:  RandomForestClassifier()  score =  0.4664654594232059

 Model:  RandomForestClassifier()  score =  0.4507042253521127

 Model:  RandomForestClassifier()  score =  0.4523809523809524

 Model:  RandomForestClassifier()  score =  0.4590878604963112

 Model:  RandomForestClassifier()  score =  0.45707578806170357

 Model:  RandomForestClassifier()  score =  0.4550637156270959

 Model:  RandomForestClassifier()  score =  0.46545942320590206

 Model:  RandomForestClassifier()  score =  0.4597585513078471

 Model:  RandomForestClassifier()  score =  0.4533869885982562

 Model:  RandomForestClassifier()  score =  0.46545942320590206

 Model:  RandomForestClassifier()  mean score =  0.45848423876592886


**Segundo Problema**. A avaliação dos diferentes hiperparâmetros dos modelos, como o valor de k no KNeighborsClassifier ou a profundidade da Árvore de Decisão, sobre o conjunto de teste traz o risco de produzirmos um sobreajuste do modelo, pois vamos empregando sempre hiperparâmetros que mais e mais elevam o ajuste do modelo ao conjunto de teste - é como se estivéssemos, de fato, empregando o conjunto de teste para o treinamento.

> *Uma analogia útil: se medir a acuracidade sobre o conjunto de treinamento é como dar uma prova para um aluno onde haveria somente questões que ele já conheceu em aula (o conjunto de treinamento), o uso do conjunto de teste para ajustar os hiperparâmetros do modelo, seria como dar a nota para o aluno (mas sem indicar as respostas) para que ele fizesse novas tentativas da prova até obter o resultado que desejasse. Em ambos os casos parece ser uma má prática, seja para o aprendizado humano, seja para o aprendizado de máquina.*

Isso poderia ser resolvido empregando-se um novo conjunto de dados normalmente chamado de **Conjunto de Validação**.

> **Conjunto de Validação** = empregado para refinar os hiperparâmetros do modelo

> **Conjunto de Teste** = empregado para avaliar o modelo final obtido

Separar mais uma porção dos dados é entretanto bastante ruim se você pensar que já separou 20%-30% para teste.  

Uma técnica comum para resolver esses dois problemas de forma conjunta é o emprego da **Validação Cruzada** que, normalmente, é quase um padrão para a seleção e ajuste de modelos sendo o emprego da separação simples de dados de treinamento e teste um ensaio apenas inicial para a construção dos modelo.

Incluíremos essa técnica no nosso pipeline de classificação.

